In [28]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
from IPython.display import Image

## Data Cleaning/Web Scraping
This first part here is going to be the data cleaning process by using beautiful soup.

First, I retrieve the contents on a page and examine them a bit.

Make a variable called `link`, that stores the following URL (as a string):

'https://archive.org/stream/GoosebumpsDeadHouse/GoosebumpsDeadHouse_djvu.txt' 

Now, to open the URL, use `requests.get()` and provide `link` as its input. Store this in a variable called `page`.

After that, make a variable called `soup` to parse the HTML using `BeautifulSoup`.

In [29]:
import requests
import bs4
from bs4 import BeautifulSoup
from tika import parser
import PyPDF2

In [30]:
link = 'https://archive.org/stream/GoosebumpsDeadHouse/GoosebumpsDeadHouse_djvu.txt' #link for the book
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')
soup

![image](Images/web_scraping.png)

### Checking Scrape Contents

Extract the text from the page and save it in a variable called `text_not_clean_1`. 

I extract it as a string.

To do so, I have to use the soup object created in the above cell. 


In [31]:
text_not_clean_1 = soup.pre.string
#print(text_not_clean_1[:500])

In [32]:
text_not_clean_2 = text_not_clean_1.replace('/n','')

In [33]:
print(text_not_clean_2[:500])

1 



Josh & I hated our new house. 

Sure, it was big. It looked like a mansion compared to our old house. It was a tall redbrick 
house with a sloping black roof & rows of windows framed by black shutters. 

It's so dark, I thought, studying it from the street. The whole house was covered in darkness, 
as if it were hiding in the shadows of the gnarled, old trees that bent over it. 

It was the middle of July, but dead brown leaves blanketed the front yard. Our sneakers 
crunched over them as 


Remove page numbers from text.

Print the sample output.

In [34]:
text = ''.join([i for i in text_not_clean_2 if not i.isdigit()])
print(text[:500])

 



Josh & I hated our new house. 

Sure, it was big. It looked like a mansion compared to our old house. It was a tall redbrick 
house with a sloping black roof & rows of windows framed by black shutters. 

It's so dark, I thought, studying it from the street. The whole house was covered in darkness, 
as if it were hiding in the shadows of the gnarled, old trees that bent over it. 

It was the middle of July, but dead brown leaves blanketed the front yard. Our sneakers 
crunched over them as w


## Creating a specification and model
The following were the specification that I set before building and traning the model:

In [8]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

embedding_dim = 256 # The embedding dimension
epochs = 50 
seq_length = 200 # Sequence length
vocab = sorted(set(text))
examples_per_epoch = len(text)//seq_length
vocab_size = len(vocab) # Length of the vocabulary in chars
rnn_units = 1024 # Number of RNN units
steps_per_epoch = examples_per_epoch//BATCH_SIZE

In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [10]:
# Create training examples / targets
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [13]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  rnn2 = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.LSTM, recurrent_activation='sigmoid')  #Use LSTM

### Constructing model

To construct the model, I use embedding as my input layer. I added two LSTM layers as well as several dropouts. For the output layer, I use a dense layer.

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    
    tf.keras.layers.Dropout(0.2),
      
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
      
    tf.keras.layers.Dropout(0.2),

    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(vocab_size)  
    ])

  return model

In [15]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  #print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           15872     
_________________________________________________________________
dropout (Dropout)            (64, None, 256)           0         
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dropout_1 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (64, None, 1024)          6297600   
_________________________________________________________________
dropout_2 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
dense (Dense)                (64, None, 62)            6

In [18]:
#sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1) # TF 1.12
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 200, 62)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1269774


In [20]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Training the model

In [22]:
EPOCHS=70

history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/70
9/9 [==============================] - 4s 485ms/step - loss: 4.6315
Epoch 2/70
9/9 [==============================] - 3s 308ms/step - loss: 3.5877
Epoch 3/70
9/9 [==============================] - 3s 308ms/step - loss: 3.1379
Epoch 4/70
9/9 [==============================] - 3s 306ms/step - loss: 3.0004
Epoch 5/70
9/9 [==============================] - 3s 302ms/step - loss: 2.8441
Epoch 6/70
9/9 [==============================] - 3s 304ms/step - loss: 2.6882
Epoch 7/70
9/9 [==============================] - 3s 306ms/step - loss: 2.5534
Epoch 8/70
9/9 [==============================] - 3s 306ms/step - loss: 2.4287
Epoch 9/70
9/9 [==============================] - 3s 314ms/step - loss: 2.3401
Epoch 10/70
9/9 [==============================] - 3s 312ms/step - loss: 2.2677
Epoch 11/70
9/9 [==============================] - 3s 304ms/step - loss: 2.2088
Epoch 12/70
9/9 [==============================] - 3s 300ms/step - loss: 2.1484
Epoch 13/70
9/9 [==============================] 

In [23]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training/ckpt_70'

In [24]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            15872     
_________________________________________________________________
dropout_3 (Dropout)          (1, None, 256)            0         
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dropout_4 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (1, None, 1024)           6297600   
_________________________________________________________________
dropout_5 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
dense_1 (Dense)              (1, None, 62)            

In [1]:
def generate_text(model, start_string):

  num_generate = 5000 

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 0.5 #set temperature

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
        
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

### Generate the output text.

Generate with 70 epochs.

(For the result of the script, some of the sentences seemed to make sense and some didn’t. Therefore, in order to create the story that makes sense, I have to manually picked the generated sentences to create the remake story. However, the originality of the generated sentences are kept.)

In [29]:
print(generate_text(model, start_string=u"Josh"))

Josh had been left it on his head. 

"Hey — that's quite a gate, I think so. Whenever he makes a big fuss about 
something, he usually get him quickly down the street. 

"Yeah, I guess," I said, thinking about Kathy & my other friends back home. 

"& Josh turned to stare at me. 

"Oh. I . . . uh . . ." His question caught me by surprise. I could hear my dad honking his horn 
impatiently down on the street. 

"Yes!" I cried. "What are you going to do?" 

His red eyes flickered, cutching a 
glowing my cheeks. The smy had ran up to the ground. 

"There's a boy upstairs. I saw him in the window. I'm not crazy," Mom said, rolling her eyes glowing with me, 
when I saw that the rows of long benches built into the living room. 

Several paint of smelled shoulder. 

"It just name on the glance of the time. I didn't like it much either of them." 

"We promised our faces & put them there?" 

I started to call out the door. The school came into focus, & I could see that he was embarrassed by how J

In [33]:
print(generate_text(model, start_string=u"This is the Dead House"))

This is the Dead House. That's why he was through the street. 

"Yes," I said, surrounded by the curtains. 

They leave a little while to figure out which way is here?" 

"We've got to find him," Mr. Dawes said, staring at me, violently motioning with her, I saw Mr. Dawes standing at the 
colder. 

It didn't turn to find our way to the street. The light had all disappeared down the hold tree. "They sound mare a little nervous. I'm not crazy, you know." 

"Says who?" Josh cracked. 

"Amanda, peered in the living room," Mr. Dawes said softly, his eyes narrowing as he reached the street. 

"Yes!" I cried, my voice shaking. I felt really nice, especially the girl 
named Karen Somerset, wanting their fates against a gravestone. 
I quickly cut in out of the gravel driveway. The night was still talk over with Mr. Dawes." 

"Yes, master," I said, turning to the morning. "I had this trem — quiet!" 

"We've got to tell Mom & Dad." Josh asked, turning to the moving down at our feet. "You ready?" 

Generate using 50 epochs.

In [61]:
print(generate_text(model, start_string=u"cemetery"))


cemetery scratching him away from the ground. "I don't feel all the car out of the garage, & then shoulders kids we didn't see anyone. 

I was beginning to be against the black hair. 

"Hey, Mr. Dawes — " I started to run away from the ground. "He doesn't believe it. 

"You weren't supposed to see miseress a bright light. 

"Where is he?" 

"It's the street before," Josh said. "It's going to be here," he said. 

"He doesn't believe it. We've been finished down the driveway with me again? 

He usually get down the driveway toward the street. "I'm not going to again?" 

"We don't know how to get him," Josh said. "What are we going to do?" 

His face was come of nothing hard. 

"Mom patter when I called up to the door. The curtains was going to do to the front of the houses on the ceiling light. 

"We've got to get away from here," Mom said along the light. I 
stopped in one house. The dog crept down on the street. 

It was the moving for Petey into the back of my looked 
barked. "The sch

In [62]:
print(generate_text(model, start_string=u"darkness"))

darkness. "You know. I'm sorry. Looking 
at at the door. "Who's there?" 

"Okay," I said, as we reached out of the dark, slifking behind him back to the car. He had a big light floated back to the car. "Who's there?" 

"I don't know what what was happened with me." 

"It's not a hard for hours." He 
called as fast as I could 
to the street. "It's a gain in the car. "Where was he 
going to be here," he said. 

"We don't know what I started to say? 

Why was he didn't see my bedroom door & growling at me. 

"I'm sorry," Mom said, as we reached the darkness. "You know. The rest of us gray light from the playground," I called. 

I was buring just on the time. I didn't believe 
the house was inside & realized this was my for him. 

"It's so dark," I said apail. 

"Josh — what's going on?" Josh said the street. "Where are you going?" 

Ray called quickly to the dead, "You want to stink on the dog come in his break & started worried by his barking. 

"Let's go inside," Mr. Dawes said to me. "

In [64]:
print(generate_text(model, start_string=u"blood"))

blood away. 

"He — it's me a few seconds, the boy in the dog," he said. "I guess you doing houre. 

"Amanda — what's going on?" Josh insisted. 

"I don't know what that would be 
gravestone. 

Mom & Dad were all of the doorknob. "I don't know what was he make behind the front of the door. 

I stopped by his face. 

"Maybe we can do it," I told him. "But I didn't see you 
that would be a pain. I was frazent you come in our front of the house. 

"Maybe we could house in our house," he said. 

"Who — what's worry." 

"We've been going to go got to get out of here?" I cried. 

We had a big breatheater, I guessed. 

"Maybe we can for a warm smile," Mr. Dawes said. 

"Who — what's worry about Petey," Josh cried. "What are you kids — " 

"Hey, what are you doing here?" 

"We've got to get to stop moving to the playground," Mr. Dawes said to me. 

"Hey— Josh!" I called. 

"Amanda — come on!" Josh whispered. "Was the thought that did I was sometimes. Josh & I both going to do it. 

I was begin

In [30]:
print(generate_text(model, start_string=u"I"))

I stared at me, vice on his face, was heading back to the ground. 

"They — they won't believe it!" Josh said, as we reached the street. "She still had a been like Dad's, & dark, & she suddenly looked different. 

Mr. Dawes had been till the boy in the window. 
"A dream?" I remembered my hot & me. "The school's just 
blocks away," he said, his voice from the halogen flashlight, I his eyes frightened. 

"Petey — stop!" Mom yelled. 

"Hey, Josh!" 

"Josh, wait up! Don't! Don't you recognize us?" 

"We've got to find them!" Josh screamed. 

"Hey, Josh!" 

He wasn't sure I dookn't realize. & to stay morning. 

"I don't remember," Josh said, staring at me, violently motioning with his house for the st time that he was embarrassed by how Josh was 
surrounded by the curtain.. 

"No. I don't think so," he said, giving Mom a reas of the time. I didn't want to see them being shoulders & took another step back both neared real steps & began dancing happily with her across the sky. 

It took me a 

In [31]:
print(generate_text(model, start_string=u"dead"))

dead whisper. 

"What are we going to do?" Josh repeated up the treet, the 
gravestones spinning in the clothes & put them away. I didn't want to see them. 

It didn't take our parents long to get us packed up & to arrange for the movers to take one long walk. & it was a smile of victory. 

 

"Nice try," Mr. Dawes said, staring straight ahead. 
"Josh! What do you want?" I screamed in a voice I didn't resist. 

"Mom! Dad!" I cried. 

"This is the Dead House. This is where everyone lives when they staring 
hard into my eyes. 

Standing this close, I could see that beneath our hands 
& then moving in on us. The curtains cast moving shadows. 

"Come on, Petey, staring out at the boy, sniffed his sneakers, & began 
snarrow, his eyes frightened, 
pleading. 

"Me, too," I said, taking Josh's hand & pulling him away from the tree. "Maybe we can do 
something." 

Josh let go of my room & quiet 
footsteps. The mughing started walking, heading down the 
long red in the center of the street. 

"S